In [56]:
import json
import os
import sys
import argparse
from collections import Counter, defaultdict
import numpy as np
import graph_tool.all as gt
import pandas as pd


src_dir = os.path.abspath(os.path.join(os.pardir, 'src'))
sys.path[0] = src_dir
from parser.support import ROLES, CREDITS
from parser.my_mongo_db_login import DB_LOGIN_INFO
import parser.support as support
import network.shift_graph_maker as sgm

In [2]:
src_dir

'/home/staff/junelee/Research-Project/movie_propagation/src'

In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [4]:
with open('../data/raw_data/movies.json') as f:
    movie_file = f.read()
    movie_data = json.loads(movie_file)

In [5]:
role = 'producing'
role_key = role + "_gender_percentage"
all_movies = support.get_movies_df(role_key)
print('Got all_movies')


Loaded IMDb movies producing_gender_percentage
Got all_movies


In [6]:
all_movies.columns
#get 90s movies with only producer, id, title, year...
movie_90s = all_movies[(all_movies.year >= 1990) & (all_movies.year < 2000)]
movie_producer_df = movie_90s[['_id', 'producers', 'producing_gender_percentage', 'title', 'year']]

In [7]:
movie_producer_df = movie_producer_df.sort_values('year')

In [12]:
movie_90s.head().reset_index()

,index,AFI match,_id,acting_all_gender_percentage,acting_credited_gender_percentage,adjusted_budget,cast_all,cast_credited,casting_directors,casting_gender_percentage,...,cinematography_gender_percentage,directing_gender_percentage,directors,genres,producers,producing_gender_percentage,screenwriters,title,writing_gender_percentage,year
0,9,1746410689,tt0105236,15.384615,16.666667,2148000.0,"[nm0000172, nm0000619, nm0000514, nm0001606, n...","[nm0000172, nm0000619, nm0000514, nm0001606, n...","[[nm0947733, main], [nm0448277, secondary], [n...",75.0,...,0.0,0.0,"[[nm0000233, director]]","[Crime, Drama, Thriller]","[[nm0004744, producer], [nm0321621, executive ...",20.0,"[[nm0000233, written by], [nm0000233, (backgro...",Reservoir Dogs,0.0,1992
1,16,1746420551,tt0109578,56.097561,56.097561,8450000.0,"[nm0000314, nm0001153, nm0662981, nm0787825, n...","[nm0000314, nm0001153, nm0662981, nm0787825, n...","[[nm0808900, main], [nm0846876, main], [nm0243...",100.0,...,0.0,0.0,"[[nm0326161, director]]","[Action, Crime, Drama, Thriller]","[[nm0038002, executive producer], [nm0324875, ...",0.0,"[[nm0307257, characters], [nm0171718, (story) ...",Death Wish V: The Face of Death,0.0,1994
2,21,1746422653,tt0109675,38.000000,38.000000,3380000.0,"[nm0005113, nm0193554, nm0000609, nm0501435, n...","[nm0005113, nm0193554, nm0000609, nm0501435, n...","[[nm0113764, main], [nm0362229, main], [nm0191...",50.0,...,0.0,0.0,"[[nm0424907, director]]",[Drama],"[[nm0071929, line producer], [nm0000490, execu...",0.0,"[[nm1302702, story], [nm0424907, (screen story...",Drop Squad,0.0,1994
3,31,1746422968,tt0116151,39.215686,43.478261,NaN,"[nm0004517, nm0001159, nm0005156, nm0000391, n...","[nm0004517, nm0001159, nm0005156, nm0000391, n...","[[nm0102699, main], [nm0002424, main], [nm0916...",75.0,...,0.0,0.0,"[[nm0477129, director]]","[Comedy, Adventure, Family]","[[nm0085542, producer], [nm0089820, co-produce...",0.0,"[[nm0394201, story], [nm0394201, (screenplay) ...",Dunston Checks In,0.0,1996
4,35,1746422944,tt0111756,16.363636,15.789474,106470000.0,"[nm0000126, nm0000598, nm0000432, nm0028625, n...","[nm0000126, nm0000598, nm0000432, nm0028625, n...","[[nm0795707, main], [nm0414018, secondary], [n...",80.0,...,0.0,0.0,"[[nm0001410, director]]","[Adventure, Biography, Crime, Drama, Western]","[[nm0000126, producer], [nm0330108, executive ...",0.0,"[[nm0330108, (written by) and], [nm0001410, wr...",Wyatt Earp,0.0,1994


In [73]:
producers = list(set([p for sublist in movie_90s['producers'].apply(lambda x: [i[1] for i in x]).tolist() for p in sublist]))

In [77]:
producers

['co-producer: Fernando Colomo P.C. (as Beatriz De La Gandara)',
 'producer (as Thomas Byrnes)',
 'associate producer (as Dean Ferrandini)',
 'executive producer (as Donna Bascom)',
 'co-producer (as Steve Dunn)',
 'co-producer (as Heather Zeegen)',
 'co-executive producer (as Richard M. Heller)',
 'line producer: Los Angeles',
 'line producer (as John J. Bruno)',
 'consulting producer (special version)',
 'associate producer (as Trek T. Kelly)',
 'executive producer: UGC Images',
 'executive producer (as George Zecevic)',
 'executive producer (as Martin Hornstein)',
 'video segment producer',
 'co-producer (as Rick Solomon)',
 'co-producer (as Ronald J. Diamond)',
 'associate producer / co-producer',
 'co-producer (as Andrew La Marca)',
 'line producer: Israel',
 'associate producer (as kevin king)',
 'line producer: French',
 'co-producer (as Richard Zinman)',
 'co-producer (as Jon Sheinberg)',
 'executive producer (as Ron Bass)',
 'producer (as Tom Wilhite)',
 'co-producer (as Zakae

In [85]:
import re

find = re.compile(r"^([^.]*)producer*")

for p in producers:
    try:
        re.search(find, p).group(0)
    except AttributeError:
        print('==============================')
        print('Attribute error')
        print(p)
        print('==============================')


Attribute error
CEO of Phoenix Pictures
Attribute error
additional photography (uncredited)
Attribute error
production executive
Attribute error
production executive: Islet
Attribute error
executive in charge of production
Attribute error
production executive: Avenue
Attribute error
studio executive
Attribute error
secretary of production
Attribute error
in association with: A Limelight Production


In [53]:
movie_90s[movie_90s['producers'].map(len) == 1]

,AFI match,_id,acting_all_gender_percentage,acting_credited_gender_percentage,adjusted_budget,cast_all,cast_credited,casting_directors,casting_gender_percentage,cinematographers,cinematography_gender_percentage,directing_gender_percentage,directors,genres,producers,producing_gender_percentage,screenwriters,title,writing_gender_percentage,year
2144,1746423478,tt0117768,25.862069,26.785714,40000000.0,"[nm0000792, nm0526649, nm0355379, nm0571327, n...","[nm0000792, nm0526649, nm0355379, nm0571327, n...","[[nm0513591, main], [nm1062205, secondary], [n...",66.666667,"[[nm0349204, cinematography]]",0.000000,0.0,"[[nm0000484, director]]","[Comedy, Adventure]","[[nm0069798, producer]]",100.0,"[[nm0550930, (characters) and], [nm0020068, ch...",The Stupids,0.000000,1996
2218,1746410361,tt0099615,26.190476,29.729730,NaN,"[nm0000008, nm0000111, nm0456124, nm0000542, n...","[nm0000008, nm0000111, nm0456124, nm0000542, n...","[[nm0272067, main], [nm0330413, main], [nm0852...",80.000000,"[[nm0005710, cinematography]]",0.000000,0.0,"[[nm0000921, director]]","[Comedy, Crime]","[[nm0516465, producer]]",0.0,"[[nm0000921, written by]]",The Freshman,0.000000,1990
2271,1746408906,tt0100134,40.000000,41.666667,NaN,"[nm0001448, nm0397124, nm0000349, nm0000563, n...","[nm0001448, nm0397124, nm0000349, nm0000563, n...","[[nm0748039, main], [nm0063482, secondary], [n...",66.666667,"[[nm0839732, cinematography]]",0.000000,0.0,"[[nm0001887, director]]",[Drama],"[[nm0000816, producer]]",0.0,"[[nm0070660, screen story], [nm0070660, (scree...",Men Don't Leave,50.000000,1990
3281,1746420388,tt0105464,33.333333,0.000000,NaN,"[nm0820544, nm0752636, nm0853808]",[nm0820544],"[[nm0671518, main]]",0.000000,"[[nm0034552, cinematography]]",0.000000,0.0,"[[nm0144301, director]]","[Crime, Drama, Mystery, Thriller]","[[nm0144301, producer]]",0.0,"[[nm0144301, writer]]",Star Time,0.000000,1992
3349,1746410691,tt0107693,80.000000,100.000000,NaN,"[nm1481421, nm1482962, nm1482986, nm1909444, n...","[nm1481421, nm1482962, nm1482986, nm1909444]",[],NaN,"[[nm0358430, cinematography]]",100.000000,100.0,"[[nm0358430, director]]",[Documentary],"[[nm0358430, producer]]",100.0,[],Nitrate Kisses,NaN,1992
8159,1746434029,tt0105128,31.034483,32.142857,14320000.0,"[nm0000411, nm0000381, nm0000317, nm0750758, n...","[nm0000411, nm0000381, nm0000317, nm0750758, n...","[[nm0080544, main], [nm0544344, main], [nm0656...",42.857143,"[[nm0005665, cinematography]]",0.000000,100.0,"[[nm0005116, director]]","[Fantasy, Horror]","[[nm0802326, producer]]",0.0,"[[nm0653693, written by]]",Pet Sematary II,0.000000,1992
9666,1746421325,tt0105729,62.337662,62.337662,NaN,"[nm0415617, nm0016047, nm0502425, nm0078136, n...","[nm0415617, nm0016047, nm0502425, nm0078136, n...",[],NaN,"[[nm0005643, cinematography]]",0.000000,0.0,"[[nm0415617, director]]",[Drama],"[[nm0938482, producer]]",100.0,"[[nm0415617, writer]]",Venice/Venice,0.000000,1992
10524,1746423557,tt0116477,20.754717,21.739130,28800000.0,"[nm0007825, nm0000277, nm0086225, nm0000306, n...","[nm0007825, nm0000277, nm0086225, nm0000306, n...","[[nm0409891, main], [nm0673227, main]]",100.000000,"[[nm0004287, cinematography]]",0.000000,0.0,"[[nm0000110, director]]",[Drama],"[[nm0057655, producer]]",0.0,"[[nm0000636, play], [nm0000110, screenplay]]",Hamlet,0.000000,1996
11816,1746435913,tt0103077,37.500000,39.130435,644000.0,"[nm0669643, nm0066138, nm0172869, nm0101033, n...","[nm0669643, nm0066138, nm0172869, nm0101033, n...",[],NaN,"[[nm0380911, cinematography]]",0.000000,0.0,"[[nm0437490, director]]","[Comedy, Horror, Sci-Fi]","[[nm0437492, producer]]",0.0,"[[nm0437490, writer]]",There's Nothing Out There,0.000000,1991
13040,1746437150,tt0118094,72.727273,100.000000,NaN,"[nm0447181, nm0024325, nm0169747, nm0178229, n...",[nm0447181],[],NaN,"[[nm0044579, cinematography]]",100.000000,100.0,"[[nm0044579, director]]",[],"[[nm0044579, producer (as Beth B.)]]",100.0,[],Visiting Desire,NaN,1996


In [54]:
class Producers(object):
    def __init__(self):
        self.node_ID = None        #Last name
        self.type = 'P'
        self.belief = None
        self.status = None
        self.role = [] #role of the producers, if the role has changed each movie
        self.movie_ids = []

    def instantiate_belief(self, belief_type, b_threshold, seeds):
        '''
        Instantiate the belief of an individual
        '''
        #Do empirical, Weiss and Wunderink are 1.0,
        # all others are 0.0
        if belief_type=='empirical':
            if self.node_ID in seeds:
                self.belief = 1.0
                self.status = 'Adopter'
            else:
                self.belief = 0.0
                self.status = 'NonAdopter'
        #Combination of empirical and random
        elif belief_type=='empirical-random':
            if self.node_ID in seeds:
                self.belief = 1.0
                self.status = 'Adopter'
            else:
                self.belief = random.uniform(0.0, b_threshold)
                self.status = 'NonAdopter'
        #Supplied by file
        elif belief_type=='apriori':
            pass
        else:
            m = "I do not recognize the method of: %s, to instantiate the belief"
            gerr.generic_error_handler(message=m)

class Movies(object):
    def __init__(self):  
        self.node_ID = None
        self.type = 'M'          
        self.title = None
        self.producers = None
        self.year = None


1

In [107]:
G = sgm.build_temporal_network(movie_90s, 'empirical', 1.0)

In [105]:
len(producers)

3171